In [98]:
import torch
pwd = '/home/zjy/project/MetaIM/data'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [99]:
import networkx as nx 
from torch_geometric.nn import MessagePassing

from torch_geometric.utils import dense_to_sparse
import torch.nn.functional as F

class SIR(MessagePassing):
    def __init__(self, N, beta, gamma, C ,steps, device):
        super().__init__(aggr="add")
        self.N = N
        self.beta = beta
        self.gamma = gamma
        self.C = C
        self.steps = steps
        self.device = device

    def init_state(self,x0=None):
        if x0 is None:
            x = torch.zeros(self.N,1).to(self.device)
            ind = torch.argsort(self.w.view(-1),descending=True)
            x[ind[:self.C]] = 1    
        else:
            x = x0    
        r = torch.zeros(self.N,1).to(self.device)
        s = 1 - x
        return s, x ,r
    
    def single_step_forward(self, edge_index, s, x, r):
        q = self.propagate(edge_index, x=torch.log(1-self.beta*x))
        q = torch.exp(q)
        s, x, r = s*q, (1-self.gamma)*x + s*(1-q) , r+ self.gamma*x
        return s, x, r

    def forward(self, edge_index, x0=None):
        s, x, r = self.init_state(x0)
        for i in range(self.steps): 
           s, x ,r = self.single_step_forward(edge_index,s,x,r)
        return s, x, r

    def message(self, x_j):
        return x_j


In [100]:
from torch_geometric.datasets import Planetoid
import torch

dataset = Planetoid(root=pwd+'/cora', name='cora')
data = dataset[0]
edge_index = data.edge_index
edge_index.size()

torch.Size([2, 10556])

In [101]:
def to_undirected(edge_index):
    """
    将有向边转换为无向边
    """
    edge_index = edge_index.t().numpy()
    # 创建一个空的集合，用于存储已添加的边
    undirected_edge_set = set()

    # 创建一个空的列表，用于存储无向边的索引
    undirected_edge_index = []

    # 遍历每一条有向边
    for edge in edge_index:
        # 将边按照节点索引排序，确保无向边的唯一性

        sorted_edge = tuple(sorted(edge))

        # 如果边没有被添加过，则将其添加到集合和列表中
        if sorted_edge not in undirected_edge_set:
            undirected_edge_set.add(sorted_edge)
            tensor_sorted_edge = torch.tensor([sorted_edge[0], sorted_edge[1]])
            undirected_edge_index.append(tensor_sorted_edge)
            undirected_edge_index.append(tensor_sorted_edge.flip(0))

    return  torch.stack(undirected_edge_index, dim=1)
u_edge_index = to_undirected(edge_index).to(device)
u_edge_index


tensor([[   0,  633,    0,  ..., 2695, 2706, 2707],
        [ 633,    0, 1862,  ..., 2694, 2707, 2706]], device='cuda:0')

In [102]:
N = data.num_nodes
beta = 0.1
gamma = 1.0
steps = 100

model = SIR(N,beta,gamma,50,steps,device).to(device)

In [103]:
individual_infection = torch.zeros(N,N)
for i in range(N):
    x0 = torch.zeros(N,1).to(device)
    x0[i] = 1
    _,x,r = model(u_edge_index, x0=x0)
    individual_infection[i] = r.squeeze().cpu()
individual_infection[0]

tensor([1.0000, 0.0028, 0.0273,  ..., 0.0000, 0.0050, 0.0156])

In [104]:
import numpy as np
np.save(pwd+'/for_meta/cora_individual_infection_sir_100.npy', individual_infection.numpy())


In [105]:
individual_infection[0][individual_infection[0] > 0.1]

tensor([1.0000, 0.1146, 0.1504, 0.1101, 0.1011, 0.1586, 0.1111, 0.1012, 0.3335,
        0.1353, 0.1171, 0.1029, 0.1494, 0.1301, 0.1519, 0.1743, 0.1840, 0.1319,
        0.1230, 0.1046, 0.1017, 0.1037, 0.2156, 0.1241, 0.1213, 0.6098, 0.1147,
        0.1054, 0.1101, 0.1108, 0.1424, 0.1867, 0.1043, 0.1451, 0.1041, 0.1602,
        0.1029, 0.1131, 0.1705, 0.1031, 0.1901, 0.1247, 0.1035, 0.1198, 0.1141,
        0.1581, 0.1213, 0.1134, 0.1179, 0.1050, 0.1323, 0.1779, 0.1379, 0.1090,
        0.1146, 0.1709, 0.1174, 0.1122, 0.1188, 0.1051, 0.1188, 0.1343, 0.1032,
        0.1178, 0.1069, 0.1064, 0.1158, 0.1051, 0.1262, 0.1194, 0.1061, 0.1136,
        0.1220, 0.1002, 0.1362, 0.1097, 0.1410, 0.3585, 0.2024, 0.1011, 0.1227,
        0.1178, 0.1576, 0.2741, 0.1088, 0.1305, 0.1084, 0.1021, 0.1413, 0.1003,
        0.1044, 0.1010, 0.1054, 0.1092, 0.1121, 0.8608, 0.1193, 0.1074, 0.1014,
        0.1011, 0.2847, 0.1031, 0.1108, 0.1240, 0.1045, 0.1031, 0.1245, 0.1062,
        0.1313, 0.3287, 0.1266, 0.3213, 

In [106]:
individual_infection[0].size()

torch.Size([2708])

5%的初始感染节点

In [107]:
seed_rate = 0.05
C = int(N * seed_rate)
C

135

In [108]:
import random
sample_num = 500

seeds_infections = torch.zeros(sample_num, 2, N)
for i in range(500):
    random.seed(i)
    random_seeds = random.sample(range(N), C)
    
    x0 = torch.zeros(N,1).to(device)
    x0[random_seeds] = 1
    _,x,r = model(u_edge_index, x0=x0)
    
    seeds_infections[i][0] = x0.squeeze().cpu()
    seeds_infections[i][1] = r.squeeze().cpu()

In [111]:
seeds_infections[0][1].sum()

tensor(283.4185)

In [113]:
np.save(pwd+'/for_meta/cora_seed_infection_sir_100.npy', seeds_infections.numpy())